In [5]:
import csv
import os
import json
import time
import random


In [6]:
with open("/home/zjusst/qms/webarena/result_stage_3_generation/flitered_triplets.csv", "r") as f:
    reader = csv.reader(f)
    # 跳过第一行
    next(reader)
    triplets = [row for row in reader]
print(len(triplets))
print(triplets[0])

1998
["('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_36.json', 0, 0, 'http://reddit.com/')", "click [20] where [20] is button 'Sort by: Hot' hasPopup: menu expanded: False", 'The user wants to sort posts by a different criterion to better organize or view the content on the homepage.', "('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_36.json', 0, 1, 'http://reddit.com/')"]


In [7]:
# import networkx as nx
# import random
# from tqdm import tqdm

# def sample_centrality_paths(graph_data, max_length=5, top_k_nodes=3, top_n_paths=0):
#     """
#     基于节点中心性采样路径。筛选高出度节点作为起点，高入度节点作为终点，
#     并寻找它们之间的重要路径。

#     Args:
#         graph_data (list of tuples): 图数据 (source, action, intent, target)。
#         max_length (int): 路径的最大长度 (cutoff)。
#         top_k_nodes (int): 筛选多少个高分节点作为候选起点/终点。
#         top_n_paths (int): 最终返回多少条得分最高的路径。

#     Returns:
#         list of list of tuples: 返回 top_n_paths 条轨迹，每条轨迹是步骤的列表。
#     """
#     if not graph_data:
#         print("错误：图数据为空。")
#         return []

#     # 1. 构建图，改成节点间不允许有平行边
#     G = nx.MultiDiGraph()
#     #G = nx.DiGraph()
#     for source, action, intent, target in graph_data:
#         G.add_edge(source, target, action=action, intent=intent)

#     # 2. 计算中心性
#     # Out-degree: 适合做起点 (Hubs)
#     out_centrality = nx.out_degree_centrality(G)
#     # In-degree: 适合做终点 (Authorities)
#     in_centrality = nx.in_degree_centrality(G)

#     # 3. 筛选 Top-K 起点和终点
#     # 按分数降序排列
#     sorted_starts = sorted(out_centrality.items(), key=lambda x: x[1], reverse=True)
#     sorted_ends = sorted(in_centrality.items(), key=lambda x: x[1], reverse=True)

#     # 取前 K 个节点的 ID
#     top_starts = [node for node, score in sorted_starts[:top_k_nodes]]
#     top_ends = [node for node, score in sorted_ends[:top_k_nodes]]

#     # print(f"筛选出的 Top-{top_k_nodes} 潜在起点: {top_starts}")
#     # print(f"筛选出的 Top-{top_k_nodes} 潜在终点: {top_ends}")

#     candidate_paths = []

#     # 创建一个 tqdm 进度条，总数为 top_starts * top_ends
#     tqdm_bar = tqdm(total=len(top_starts) * len(top_ends), desc="寻找路径")
#     # 4. 寻找起点和终点之间的路径
#     for start_node in top_starts:
#         for end_node in top_ends:
#             if start_node == end_node:
#                 continue
            
#             # 修改策略：不再找所有路径，而是找由短到长的路径，并限制数量
#             try:
#                 # 这是一个生成器，它会按路径长度递增的顺序 yield 路径
#                 path_generator = nx.shortest_simple_paths(G, source=start_node, target=end_node)
                
#                 paths_found_for_pair = 0
#                 max_paths_per_pair = 5  # 限制每一对节点最多只取 5 条
                
#                 for nodes_path in path_generator:
#                     # 1. 检查长度限制 (cutoff)
#                     # shortest_simple_paths 不接受 cutoff 参数，所以我们需要手动检查
#                     # 注意：len(nodes_path) 是节点数，边数是 len - 1
#                     if len(nodes_path) - 1 > max_length:
#                         break # 因为后面生成的只会更长，所以直接停止
                    
#                     # 2. 过滤过短路径
#                     if len(nodes_path) < 2:
#                         continue

#                     # 3. 计算评分并加入候选列表
#                     # 依然使用中心性评分，这样在全局筛选时，重要的路径会排在前面
#                     path_score = sum(in_centrality[n] + out_centrality[n] for n in nodes_path) / len(nodes_path)
                    
#                     candidate_paths.append({
#                         "score": path_score,
#                         "nodes": nodes_path
#                     })
                    
#                     # 4. 计数控制
#                     paths_found_for_pair += 1
#                     if paths_found_for_pair >= max_paths_per_pair:
#                         break # 该对节点已找到足够的路径，跳出循环，处理下一对节点

#             except nx.NetworkXNoPath:
#                 continue
#             except nx.NodeNotFound:
#                 continue

#             # 5. 对每条路径进行评分
#             for nodes_path in paths:
#                 # 过滤掉太短的路径（例如只有1步的），根据需要调整
#                 if len(nodes_path) < 2: 
#                     continue

#                 # 评分逻辑：路径上所有节点的 (入度分 + 出度分) 之和 / 路径长度
#                 # 这里可以自定义，比如只看终点的入度，或者看关键节点的介数中心性
#                 path_score = sum(in_centrality[n] + out_centrality[n] for n in nodes_path) / len(nodes_path)
                
#                 candidate_paths.append({
#                     "score": path_score,
#                     "nodes": nodes_path
#                 })
#             tqdm_bar.update(1)

#     # 6. 排序并选择 Top-N 路径
#     # 按分数降序排序
#     candidate_paths.sort(key=lambda x: x["score"], reverse=True)

#     if top_n_paths == 0:
#         top_n_paths = min(len(candidate_paths), 5000)

#     print(f"总数: {len(candidate_paths)}，选择: {top_n_paths}")
#     best_paths_data = candidate_paths[:top_n_paths]

#     final_trajectories = []

#     # 7. 还原路径详情 (补充 Action 和 Intent)
#     for item in best_paths_data:
#         nodes = item["nodes"]
#         trajectory = []
        
#         # 遍历路径中的节点对 (u, v)
#         for i in range(len(nodes) - 1):
#             u = nodes[i]
#             v = nodes[i+1]
            
#             # 获取 u 到 v 的所有边
#             # G[u][v] 返回一个字典，key 是边的索引，value 是属性字典
#             all_edges = G[u][v]
            
#             # 策略：如果有平行边（多条动作指向同一页面），这里随机选一条
#             # 也可以改为选择 intent 看起来最有意义的一条
#             chosen_key = random.choice(list(all_edges.keys()))
#             edge_data = all_edges[chosen_key]
            
#             step_tuple = (u, edge_data['action'], edge_data['intent'], v)
#             trajectory.append(step_tuple)
            
#         final_trajectories.append(trajectory)

#     print(f"已生成 {len(final_trajectories)} 条关键路径。")
#     return final_trajectories

# # ==================== 示例用法 ====================
# if __name__ == "__main__":
#     # 构造一个稍微复杂一点的图来演示中心性
#     # PageMain 是核心起点（高出度）
#     # PageCheckout 是核心终点（高入度）
#     graph_tuples = triplets
    
#     # 期望找出从 Main 到 Checkout 的路径
#     print("--- 基于中心性的关键路径采样 ---")
    
#     # 寻找最长5步，Top 2 起点/终点，返回 Top 3 路径
#     key_trajectories = sample_centrality_paths(
#         graph_tuples, 
#         max_length=5, 
#         top_k_nodes=10, 
#         top_n_paths=0
#     )

In [ ]:
import networkx as nx
# import random # DiGraph 不需要随机选择平行边了

def sample_centrality_paths_digraph(graph_data, max_length=5, top_k_nodes=3, top_n_paths=0):
    if not graph_data:
        print("错误：图数据为空。")
        return []

    # Change 1: 使用 DiGraph
    G = nx.DiGraph() 
    for source, action, intent, target in graph_data:
        # 注意：如果有平行边，这里会发生覆盖！只有最后一个动作会被保留
        G.add_edge(source, target, action=action, intent=intent)

    # ... [中间计算中心性、筛选节点的代码完全不用变] ...
    # Out-degree: 适合做起点 (Hubs)
    out_centrality = nx.out_degree_centrality(G)
    in_centrality = nx.in_degree_centrality(G)

    sorted_starts = sorted(out_centrality.items(), key=lambda x: x[1], reverse=True)
    sorted_ends = sorted(in_centrality.items(), key=lambda x: x[1], reverse=True)

    top_starts = [node for node, score in sorted_starts[:top_k_nodes]]
    top_ends = [node for node, score in sorted_ends[:top_k_nodes]]
    
    candidate_paths = []

    tqdm_bar = tqdm(total=len(top_starts) * len(top_ends), desc="寻找路径")

    for start_node in top_starts:
        for end_node in top_ends:
            if start_node == end_node:
                continue
            
            # 修改策略：不再找所有路径，而是找由短到长的路径，并限制数量
            try:
                # 这是一个生成器，它会按路径长度递增的顺序 yield 路径
                path_generator = nx.shortest_simple_paths(G, source=start_node, target=end_node)
                
                paths_found_for_pair = 0
                max_paths_per_pair = 5  # 限制每一对节点最多只取 5 条
                
                for nodes_path in path_generator:
                    # 1. 检查长度限制 (cutoff)
                    # shortest_simple_paths 不接受 cutoff 参数，所以我们需要手动检查
                    # 注意：len(nodes_path) 是节点数，边数是 len - 1
                    if len(nodes_path) - 1 > max_length:
                        break # 因为后面生成的只会更长，所以直接停止
                    
                    # 2. 过滤过短路径
                    if len(nodes_path) < 2:
                        continue

                    # 3. 计算评分并加入候选列表
                    # 依然使用中心性评分，这样在全局筛选时，重要的路径会排在前面
                    path_score = sum(in_centrality[n] + out_centrality[n] for n in nodes_path) / len(nodes_path)
                    
                    candidate_paths.append({
                        "score": path_score,
                        "nodes": nodes_path
                    })
                    
                    # 4. 计数控制
                    paths_found_for_pair += 1
                    if paths_found_for_pair >= max_paths_per_pair:
                        break # 该对节点已找到足够的路径，跳出循环，处理下一对节点

            except nx.NetworkXNoPath:
                continue
            except nx.NodeNotFound:
                continue

            tqdm_bar.update(1)

    candidate_paths.sort(key=lambda x: x["score"], reverse=True)
    
    if top_n_paths == 0:
        top_n_paths = min(len(candidate_paths), 5000)

    print(f"总数: {len(candidate_paths)}，选择: {top_n_paths}")
    best_paths_data = candidate_paths[:top_n_paths]

    final_trajectories = []

    # Change 2: 还原路径详情 (核心修改)
    for item in best_paths_data:
        nodes = item["nodes"]
        trajectory = []
        
        for i in range(len(nodes) - 1):
            u = nodes[i]
            v = nodes[i+1]
            
            # --- 修改开始 ---
            # 在 DiGraph 中，两个节点间只有一条边，直接获取属性
            edge_data = G[u][v] 
            
            # 不需要再做 random.choice 了，因为只有这一条边
            # --- 修改结束 ---
            
            step_tuple = (u, edge_data['action'], edge_data['intent'], v)
            trajectory.append(step_tuple)
            
        final_trajectories.append(trajectory)

    return final_trajectories


# ==================== 示例用法 ====================
if __name__ == "__main__":
    graph_tuples = triplets
    
    # 期望找出从 Main 到 Checkout 的路径
    print("--- 基于中心性的关键路径采样 ---")
    
    # 寻找最长5步，Top 2 起点/终点，返回 Top 3 路径
    key_trajectories = sample_centrality_paths_digraph(
        graph_tuples, 
        max_length=5, 
        top_k_nodes=100, 
        top_n_paths=0
    )

--- 基于中心性的关键路径采样 ---


寻找路径:  94%|█████████▍| 9435/10000 [00:01<00:00, 5210.83it/s]

总数: 20514，选择: 5000


In [ ]:
# 统计 key_trajectories 每条路径的长度的分布
length_distribution = {}
for traj in key_trajectories:
    length = len(traj)
    if length not in length_distribution:
        length_distribution[length] = 0
    length_distribution[length] += 1

# 打印长度分布
print("\n路径长度分布:")
for length, count in sorted(length_distribution.items()):
    print(f"{length} 步: {count} 条")





路径长度分布:
1 步: 72 条
2 步: 449 条
3 步: 1418 条
4 步: 2084 条
5 步: 977 条


In [17]:
import random

out_list = []

# key_trajectories 中，长度为 2，3，4，5，各采集前 17 条
count_dict = {
    2: 0,
    3: 0,
    4: 0,
    5: 0
}

for traj in key_trajectories:
    traj_len = len(traj)
    if traj_len in count_dict and count_dict[traj_len] < 17:
        out_list.append({
            "selected_trajectory": traj
        })
        count_dict[traj_len] += 1

print(f"共采集到 {len(out_list)} 条关键路径")
for l in [2, 3, 4, 5]:
    print(f"长度为 {l} 的路径数量：{count_dict[l]}")


共采集到 68 条关键路径
长度为 2 的路径数量：17
长度为 3 的路径数量：17
长度为 4 的路径数量：17
长度为 5 的路径数量：17


In [18]:
# 将 out_list 导出到 json 文件, /home/zjusst/qms/webarena/result_stage_3_generation/random_selected_trajs.json
with open("/home/zjusst/qms/webarena/result_stage_3_generation/selected_importance_trajs.json", "w") as f:
    json.dump(out_list, f)
